In [7]:
'''
file_id_mesgs: general info about Garmin device and activity
file_creator_mesgs: software version
event_mesgs: general info about event? (ex. alert for Garmin live track?)
device_info_mesgs: info of all devices and sensors owned
device_settings_mesgs: time and specific device settings
user_profile_mesgs: user profile info and settings
sport_mesgs: activity sport
zones_target_mesgs: HR and Power tresholds

record_mesgs: all activity info!!!

lap_mesgs: lap data
time_in_zone_mesgs: HR and Power zones spent during the activity
session_mesgs: training session info recap
activity_mesgs: activity brief recap
'''

from garmin_fit_sdk import Decoder, Stream
import folium
import dash
import dash_bootstrap_components as dbc
import math
import numpy as np
from dash import Dash, html

mapFileName = "route_map.html"

class Position:
    SEMICIRCLE_TO_DEGREE = 180 / (2**31)

    def __init__(self, latitude: float, longitude: float):
        self.latitude = self.semicircles_to_degrees(latitude)
        self.longitude = self.semicircles_to_degrees(longitude)

    @staticmethod
    def semicircles_to_degrees(value: float) -> float:
        return value * Position.SEMICIRCLE_TO_DEGREE

    def __repr__(self):
        return f"Position(latitude={self.latitude}, longitude={self.longitude})"

class ActivityData:
    def __init__(self, name, value, unit_of_measurement, icon):
        self.name = name
        self.value = value
        self.unit_of_measurement = unit_of_measurement
        self.icon = icon

    def __repr__(self):
        return f"ActivityData(name={self.name}, value={self.value}, unit_of_measurement={self.unit_of_measurement})"

def find_center(coordinates):
    if not coordinates:
        return None
    x_center = sum(x for x, y in coordinates) / len(coordinates)
    y_center = sum(y for x, y in coordinates) / len(coordinates)
    return (x_center, y_center)

def parse_fit_file(filepath):
    stream = Stream.from_file(filepath)
    decoder = Decoder(stream)
    messages, errors = decoder.read()
    if errors:
        print(f"Errors found while decoding FIT file: {errors}")
    return messages

def extract_coordinates(record_messages):
    coordinates = []
    position_lat = "position_lat"
    position_long = "position_long"
    for message in record_messages:
        if position_lat in message and position_long in message:
            coordinates.append(
                Position(message[position_lat], message[position_long])
            )
    return coordinates

def create_and_save_route_map(coordinates, output_file="map.html"):
    folium_coordinates = [[coord.latitude, coord.longitude] for coord in coordinates]
    map_center = find_center(folium_coordinates)
    start = (coordinates[0].latitude, coordinates[0].longitude)
    end = (coordinates[-1].latitude, coordinates[-1].longitude)
    m = folium.Map(location=map_center, zoom_start=11)
    folium.Marker(location=start, tooltip="Start", popup="Home sweet home", icon=folium.Icon(color="green")).add_to(m)
    folium.Marker(location=end, tooltip="End", popup="Home sweet home", icon=folium.Icon(color="red")).add_to(m)
    folium.PolyLine(folium_coordinates, tooltip="route").add_to(m)
    m.save(output_file)

def generate_dashboard(data):
    rows = []
    for i in range(0, len(data), 4):
        row_columns = []
        for j in range(i, min(i + 4, len(data))):
            stat = data[j]
            column = dbc.Col(
                html.Div([
                    html.I(className=stat.icon, style={"font-size": "24px", "margin-bottom": "10px", "color": "#007bff"}),
                    html.H4(stat.name),
                    html.H6(f"{stat.value} {stat.unit_of_measurement}")
                ], style={"border": "1px solid #ddd", "padding": "10px", "text-align": "center", "border-radius": "5px"}),
                width=3  # Each column takes 1/3 of the row width
            )
            row_columns.append(column)
        rows.append(dbc.Row(row_columns, className="mb-3"))
    return rows

def get_activity_duration(total_seconds):
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    return hours, minutes, seconds

def get_activity_duration_as_string(total_seconds):
    hours, minutes, seconds = get_activity_duration(total_seconds)
    return f"{hours:02}:{minutes:02}:{seconds:02}"

def haversine(coord1, coord2):
    R = 6371  # Radius of the Earth in kilometers
    lat1, lon1 = map(math.radians, [coord1.latitude, coord1.longitude])
    lat2, lon2 = map(math.radians, [coord2.latitude, coord2.longitude])
    delta_lat, delta_lon = lat2 - lat1, lon2 - lon1
    a = math.sin(delta_lat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(delta_lon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def total_distance(positions):
    return round(sum(haversine(positions[i], positions[i + 1]) for i in range(len(positions) - 1)), 2)

def get_positive_ascent(activity_data, altitude_key):
    return round(sum(
        activity_data[i + 1][altitude_key] - activity_data[i][altitude_key]
        for i in range(len(activity_data) - 1)
        if altitude_key in activity_data[i] and altitude_key in activity_data[i + 1] and activity_data[i + 1][altitude_key] > activity_data[i][altitude_key]
    ), 2)

def get_average_parameter(activity_data, parameter_key):
    valid_data = [message[parameter_key] for message in activity_data if parameter_key in message]
    return round(sum(valid_data) / len(valid_data), 2) if valid_data else 0

def get_max_parameter(activity_data, parameter_key):
    return max((message.get(parameter_key, float('-inf')) for message in activity_data), default=float('-inf'))

def get_max_parameter_in_minutes(activity_data, parameter_key, interval_in_minutes):
    interval_in_seconds = interval_in_minutes * 60
    max_average = 0
    start_time, end_time = 0, 0
    for i in range(len(activity_data) - interval_in_seconds):
        segment = activity_data[i:i + interval_in_seconds]
        valid_values = [msg[parameter_key] for msg in segment if parameter_key in msg]
        if valid_values:
            average_value = sum(valid_values) / len(valid_values)
            if average_value > max_average:
                max_average = average_value
                start_time, end_time = i, i + interval_in_seconds
    return round(max_average, 2), get_activity_duration_as_string(start_time), get_activity_duration_as_string(end_time)

def calculate_all_activity_data(raw_activity_data):
    all_activity_data = []

    activity_duration_seconds = len(raw_activity_data)
    distance = total_distance(extract_coordinates(raw_activity_data))    
    average_power = int(get_average_parameter(raw_activity_data, "power"))
    max_20_min_speed_m_s, max_20_min_speed_start_time, max_20_min_speed_end_time = get_max_parameter_in_minutes(raw_activity_data, "speed", 20)
    
    all_activity_data.append(ActivityData("Time in motion", get_activity_duration_as_string(activity_duration_seconds), "", "fa-solid fa-clock"))
    all_activity_data.append(ActivityData("Distance", distance, "Km", "fa-solid fa-road"))
    all_activity_data.append(ActivityData("Kilojoule", int(average_power * activity_duration_seconds / 1000), "kJ", "fa-solid fa-fire"))

    all_activity_data.append(ActivityData("Average heart rate", int(get_average_parameter(raw_activity_data, "heart_rate")), "BPM", "fa-solid fa-heart"))
    all_activity_data.append(ActivityData("Max heart rate", int(get_max_parameter(raw_activity_data, "heart_rate")), "BPM", "fa-solid fa-heart"))
    all_activity_data.append(ActivityData("Max heart rate (20 minutes)", int(get_max_parameter_in_minutes(raw_activity_data, "heart_rate", 20)[0]), "BPM", "fa-solid fa-heart"))    

    all_activity_data.append(ActivityData("Average speed", round(distance / (activity_duration_seconds / 3600), 2), "Km/h", "fa-solid fa-gauge-high"))
    all_activity_data.append(ActivityData("Max speed", round(get_max_parameter(raw_activity_data, "speed") * 3.6, 3), "km/h", "fa-solid fa-gauge-high"))
    all_activity_data.append(ActivityData("Max speed (20 minutes)", max_20_min_speed_m_s * 3.6, "km/h", "fa-solid fa-gauge-high"))

    all_activity_data.append(ActivityData("Average power", average_power, "W", "fa-solid fa-bolt"))
    all_activity_data.append(ActivityData("Normalized power", get_normalized_power(raw_activity_data), "W", "fa-solid fa-bolt"))
    all_activity_data.append(ActivityData("Max power", get_max_parameter(raw_activity_data, "power"), "W", "fa-solid fa-bolt"))
    all_activity_data.append(ActivityData("Max power (20 minutes)", int(get_max_parameter_in_minutes(raw_activity_data, "power", 20)[0]), "W", "fa-solid fa-bolt"))    

    all_activity_data.append(ActivityData("Average cadence", int(get_average_parameter(raw_activity_data, "cadence")), "RPM", "fa-solid fa-rotate-right"))
    all_activity_data.append(ActivityData("Max cadence", int(get_max_parameter(raw_activity_data, "cadence")), "RPM", "fa-solid fa-rotate-right"))
    all_activity_data.append(ActivityData("Max cadence (20 minutes)", int(get_max_parameter_in_minutes(raw_activity_data, "cadence", 20)[0]), "RPM", "fa-solid fa-rotate-right"))

    all_activity_data.append(ActivityData("Positive elevation", get_positive_ascent(raw_activity_data, "altitude"), "m", "fa-solid fa-mountain"))
    all_activity_data.append(ActivityData("Max altitude", get_max_parameter(raw_activity_data, "altitude"), "m", "fa-solid fa-mountain"))
    
    
    return all_activity_data

def showDashboard(all_activity_data):
    app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0-beta3/css/all.min.css"])
    
    app.layout = html.Div([
        html.H1("Overview", style={"text-align": "center", "margin-bottom": "20px"}),
        html.Div(generate_dashboard(all_activity_data)),
        html.Iframe(
            srcDoc=open(mapFileName, "r").read(),
            width="100%",
            height="600px",
            style={"margin-top": "20px"}
        )
    ], style={"margin": "20px"})
    
    if __name__ == "__main__":
        app.run_server(debug=True)    

def get_normalized_power(raw_data):
    raw_power_data = [entry.get("power", 0) for entry in raw_data]

    normalized_power_segments = [
        sum(raw_power_data[i:i + 30])**4 / 30**4
        for i in range(0, len(raw_power_data), 30)
    ]

    normalized_power = (sum(normalized_power_segments) / len(normalized_power_segments))**(1/4)
    
    return int(normalized_power)
    

def main():
    filepath = '../data/road_cycling_activity.fit'
    messages = parse_fit_file(filepath)
    record_mesgs = messages["record_mesgs"]
    session_mesgs = messages["session_mesgs"][0]
    coordinates = extract_coordinates(record_mesgs)
    all_activity_data = calculate_all_activity_data(record_mesgs)  
    all_activity_data.append(ActivityData("Calories", session_mesgs["total_calories"], "kcal", "fa-solid fa-cookie-bite"))
    all_activity_data.append(ActivityData("Total time spent", get_activity_duration_as_string(int(session_mesgs["total_elapsed_time"])), "", "fa-solid fa-clock"))
    create_and_save_route_map(coordinates, mapFileName)
    showDashboard(all_activity_data)    

main()
